## Monad test out
Today, I ran into `monad` while reading Matthew Rocklin's blog: "Write Dumb Code" https://matthewrocklin.com/write-dumb-code.html, where he mentions as you progress as developer, you simplify the code that finally leads you to the monad. I got interested and looked it up. And found this blog by Philip Williams:
- https://www.philliams.com/monads-in-python/
This is for testing out the codes.

In monad, `unit` function corresponds to the initializer.
<br> And there's `bind` function that describes the monad's behavior, which returns the object itself.

In [2]:
from collections.abc import Callable


class MaybeMonad:
    def __init__(self, value: object = None, contains_value: bool = True):
        self.value = value
        self.contains_value = contains_value
    
    def bind(self, f: Callable) -> 'MaybeMonad':
        if not self.contains_value:
            return MaybeMonad(None, contains_value=False)  # returns empty object
        try:
            result = f(self.value)     # execute the bind function, if no value, it'll throw up
            return MaybeMonad(result)  # if success, return the result value as another monad
        except:
            return MaybeMonad(None, contains_value=False)  # gracefully exits for the failing case

Now, how to use this `Maybe Monad` to make a bunch of computations...

In [3]:
import numpy as np
# maybe_monad import MaybeMonad

value = 100
m1 = MaybeMonad(value)
print(m1.value)
print(m1.contains_value)

100
True


In [4]:
m2 = m1.bind(np.sqrt)   # typical pattern of functional programming.. chain up operations
print(m2.value)

10.0


In [5]:
m3 = m2.bind(lambda x : x / 0)   # divide by zero
print(m3.contains_value)
print(m3.value)

True
inf


/tmp/ipykernel_1063720/1906283065.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  m3 = m2.bind(lambda x : x / 0)   # divide by zero


In [6]:
def exc(x):
    raise Exception('Failed')
    
m4 = m3.bind(exc)
print(m4.contains_value)
print(m4.value)

False
None


Now, with exc bind function, we can catch the exception, and keep the state of value as None.
<br> IOW, all of try-catch logic is captured inside of `bind` method, and we can stream line the operations w/o worrying about exceptional cases. 

## Failure Monad

In [7]:
from collections.abc import Callable
from typing import Dict
import traceback

class FailureMonad:
    def __init__(self, value: object = None, error_status: Dict = None):
        self.value = value
        self.error_status = error_status
        
    def bind(self, f: Callable, *args, **kwargs) -> 'FailureMonad':   # note the pattern of returning self
        if self.error_status:
            return FailureMonad(None, error_status=self.error_status)  # when error, return that error
        
        # else call the function
        try:
            result = f(self.value, *args, **kwargs)
            return FailureMonad(result)
        
        except Exception as e:
            failure_status = {
                'trace': traceback.format_exc(),
                'exc': e,
                'args': args,
                'kwargs': kwargs
            }
            
            return FailureMonad(None, error_status=failure_status)   # return details of error
        

Now, test it out:

In [8]:
import numpy as np
#from failure_monad import FailureMonad

def dummy_func(a, b, c=3):
    return a + b + c

def exc(x):
    raise Exception('Failed')
    
value = 100
m1 = FailureMonad(value)
print(m1.value)

m2 = m1.bind(np.sqrt)
print(m2.value)

m3 = m2.bind(dummy_func, 1, c=2)
print(m3.value)     # 13.0

m4 = m3.bind(exc)
print(m4.value)  # None
print(m4.error_status)

100
10.0
13.0
None
{'trace': 'Traceback (most recent call last):\n  File "/tmp/ipykernel_1063720/1657185084.py", line 16, in bind\n    result = f(self.value, *args, **kwargs)\n  File "/tmp/ipykernel_1063720/3988368452.py", line 8, in exc\n    raise Exception(\'Failed\')\nException: Failed\n', 'exc': Exception('Failed'), 'args': (), 'kwargs': {}}


In above, `exc` bind function fails due to missing positional argument `b`. Even with this exception, code sequence proceeds.
Also, you can easily add up `bind` functions to handle different exceptions:
```
m = FailureMonad(...)

m = m.bind(func_1)
m = m.bind(func_2)
m = m.bind(func_3)

if m.error_status:
    e = m.error_status['exc']   # if exception is non-null
    if isinstance(e, ...):
        do_something()
    elif isinstance(e, ...):
        do_something_else()
    else:
        do_thing()
```

## The Lazy Monad
Some useful way to modify the behavior of the program, i.e. creating a lazily evaluated pipeline of operations.

In [11]:
class LazyMonad:
    def __init__(self, value: object):
        if isinstance(value, Callable):  # if the value is function, store the function
            self.compute = value
        else:
            def return_val():
                return value
            self.compute = return_val    # if number, store the function that returns value
                                         # when these are chained, it will become lazy evaluation
    
    def bind(self, f: Callable, *args, **kwargs) -> 'LazyMonad':
        def f_compute():
            return f(self.compute(), *args, **kwargs)
        
        return LazyMonad(f_compute)

In [12]:
import numpy as np
# from lazy_monad import LazyMonad

def dummy_func1(e):
    print(f'dummy_1 : {e}')
    return e

def dummy_func2(e):
    print(f'dummy_2 : {e}')
    return e

def dummy_func3(e):
    print(f'dummy_3 : {e}')
    return e

print('Start')

value = 100
m1 = LazyMonad(value)
print('After init')

m2 = m1.bind(dummy_func1)
print('After 1')

m3 = m2.bind(dummy_func2)
print('After 2')

m4 = m3.bind(dummy_func3)
print('After 3')

print(m4.compute())

print('After Compute')


Start
After init
After 1
After 2
After 3
dummy_1 : 100
dummy_2 : 100
dummy_3 : 100
100
After Compute


With actual computations, this is exactly what `Dask` is doing with delayed computation.

In [14]:
from time import sleep

def increment(e):
    sleep(1)
    return e + 1

def add(a, b):
    sleep(2)
    return a + b

def double(e):
    sleep(3)
    return e * 2

print('Start')

x = 100
m1 = LazyMonad(x)
print('After init')

m2 = m1.bind(increment)
print('After 1')  # 101

m3 = m2.bind(add, 6)
print('After 2')  # 107

m4 = m3.bind(double)
print('After 3')  # 214

print(m4.compute())

# after 6 seconds
print('After Compute')

Start
After init
After 1
After 2
After 3
214
After Compute


### Take-away
- Unit : Take a value and turn it into a Monad
- Bind : Take a Monad, apply a function to it and return a new Monad

### Ref
- Eric Lippert's blog: https://ericlippert.com/2013/02/21/monads-part-one/
- Arit Bhargava: "Functors, Applicatives and Monads in Pictures" https://adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html
- O'Reilly book: Real World Haskell, Bryan O'Sullivan